##### PART - I #####

In [1710]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import spacy
import numpy as np
from spacy.en import English
import string
import unicodedata

from collections import Counter
from itertools import chain

parser = English()



sqd_data = pd.read_json('/home/anjana/Downloads/train-v1.1.json') 

##### Prepare Stopwords list, question type mapping list, punctuation mark list #####

In [1711]:
stopword = stopwords.words('english')
qtntype_list = ['What', 'When', 'Why', 'Which', 'Who','Whom', 'How']
pun_list = list(string.punctuation)
delelements_list = stopword + qtntype_list + pun_list
 

##### Considering one matter from SQuAD data set - (Zeroth matter) ##### 

In [1712]:
matter = sqd_data.data[0]['paragraphs'][0] 
matter

{u'context': u'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 u'qas': [{u'answers': [{u'answer_start': 515,
     u'text': u'Saint Bernadette Soubirous'}],
   u'id': u'5733be284776f41900661182',
   u'question': u'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'},
  {u'answers': [{u'answer_start': 188, u'text': u'a copper s

##### Extraction of  context #####

In [1713]:
context = sqd_data.data[0]['paragraphs'][0]['context'] 
context = unicodedata.normalize('NFKD', context).encode('ascii','ignore')
context


'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'

##### Context pre processing ##### 

###### Sentence tokenization of the context #####

In [1714]:
senttokenized_context = sent_tokenize(context)
sentencelist = []
for i in senttokenized_context:
      sentencelist.append(i) 

In [1715]:
senttokenized_context

['Architecturally, the school has a Catholic character.',
 "Atop the Main Building's gold dome is a golden statue of the Virgin Mary.",
 'Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes".',
 'Next to the Main Building is the Basilica of the Sacred Heart.',
 'Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection.',
 'It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858.',
 'At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.']

##### Word tokenization, Lemmatisation, stop word removal #####

In [1716]:
sentence_list = []
lemmatizer = WordNetLemmatizer()


for i in sentencelist:
    words = nltk.word_tokenize(i)
    words = ([x.lower() for x in words])
    words = ([x for x in words if x not in delelements_list])
    words = [lemmatizer.lemmatize(x) for x in words]
    words = [[s.encode('ascii') for s in words]]
    sentence_list.append(words)

    

##### Questions list ##### 

In [1717]:
question_list = []
for i in range(len(sqd_data.data[0]['paragraphs'][0]['qas'])): 
    l1 = sqd_data.data[0]['paragraphs'][0]['qas'][i]['question'] 
    l1 = unicodedata.normalize('NFKD', l1).encode('ascii','ignore')

    question_list.append(l1) 

In [1718]:
question_list[0] 

'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'

* Keeping question's first word in upper case 

In [1719]:
import re

def replace_all(repls, str):
    # return re.sub('|'.join(repls.keys()), lambda k: repls[k.group(0)], str)                                     
    return re.sub('|'.join(re.escape(key) for key in repls.keys()),
                  lambda k: repls[k.group(0)], str)     

In [1720]:
Question_list = []
for i in range(len(question_list)): 
      Question_list.append(replace_all({"how": "How", "when": "When","what": "What","whom": "Whom","which": "Which"}, question_list[i]) 
                          )

In [1721]:
Question_list

['To Whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'What is in front of the Notre Dame Main Building?',
 'The Basilica of the Sacred heart at Notre Dame is beside to Which structure?',
 'What is the Grotto at Notre Dame?',
 'What sits on top of the Main Building at Notre Dame?']

##### Mapping of questions #####

In [1722]:
question_mapping= []
delquestionpart_list = []

for i in range(len(Question_list)):
    l2 = nltk.word_tokenize(Question_list[i])
    for i in range(len(l2)):
        if l2[i] in qtntype_list:
                combine_list = l2[i]+" "+l2[i+1]
                remove_list = combine_list.split() 
                l3 = [x for x in l2 if x not in remove_list]
                question_mapping.append(combine_list) 
                delquestionpart_list.append(l3) 

In [1723]:
question_mapping

['Whom did', 'What is', 'Which structure', 'What is', 'What sits']

##### Converting question sentence into 'unicode' structure for NER #####

In [1724]:
joined_delquestionpart_list =  []
for i in range(len(delquestionpart_list)):
        joined_question = unicode(" ".join(delquestionpart_list[i]),"utf-8") 
        joined_delquestionpart_list.append(joined_question) 

In [1725]:
joined_delquestionpart_list


[u'To the Virgin Mary allegedly appear in 1858 in Lourdes France ?',
 u'in front of the Notre Dame Main Building ?',
 u'The Basilica of the Sacred heart at Notre Dame is beside to ?',
 u'the Grotto at Notre Dame ?',
 u'on top of the Main Building at Notre Dame ?']

In [1726]:
import spacy
nlp = spacy.load('en')

qn_ner_mapping = []
for i in range(len(joined_delquestionpart_list)):
    doc = nlp(joined_delquestionpart_list[i])
    for ent in doc.ents:
        qn_ner_mapping.append([i,ent.text,ent.label_])  

In [1727]:
qn_ner_mapping

[[0, u'1858', u'DATE'],
 [0, u'Lourdes France', u'GPE'],
 [1, u'the Notre Dame Main Building', u'LOC'],
 [2, u'Basilica', u'PERSON'],
 [2, u'Notre Dame', u'GPE'],
 [3, u'Notre Dame', u'GPE'],
 [4, u'the Main Building', u'FAC']]

##### Data frame for QUESTION mapping #####

In [1728]:
Question_no = []
Qn_word = [] 
Qn_tag = []
for i in range(len(qn_ner_mapping)):
    Question_no.append(qn_ner_mapping[i][0])
    Qn_word.append(qn_ner_mapping[i][1])
    Qn_tag.append(qn_ner_mapping[i][2])
     

In [1729]:
question_mapping

['Whom did', 'What is', 'Which structure', 'What is', 'What sits']

In [1730]:
for i in range(len(question_mapping)):
    df = question_mapping.index(question_mapping[i])
    print(df)

0
1
2
1
4


In [1731]:
Question_character_df = pd.DataFrame()
qn_mapping = []
for i in range(len(Question_no)):
    for j in range(len(question_mapping)):
        if (Question_no[i]) == j:
            qn_mapping.append(question_mapping[j])


Question_character_df['QuestionNO'] = Question_no
Question_character_df['QuestionMapping'] = qn_mapping 
Question_character_df['Word'] = Qn_word
Question_character_df['WordTag'] = Qn_tag


##### Question - mapping #####

In [1732]:
Question_character_df

,QuestionNO,QuestionMapping,Word,WordTag
0,0,Whom did,1858,DATE
1,0,Whom did,Lourdes France,GPE
2,1,What is,the Notre Dame Main Building,LOC
3,2,Which structure,Basilica,PERSON
4,2,Which structure,Notre Dame,GPE
5,3,What is,Notre Dame,GPE
6,4,What sits,the Main Building,FAC


################################################################################################################

##### Context preprocessing ##### 

In [1733]:
context = sqd_data.data[0]['paragraphs'][0]['context'] 
context = unicodedata.normalize('NFKD', context).encode('ascii','ignore')
context

'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'

##### Sentence tokenization #####

In [1734]:
senttokenized_context = sent_tokenize(context)
sentencelist = []
for i in senttokenized_context:
      sentencelist.append(i) 

In [1735]:
senttokenized_context


['Architecturally, the school has a Catholic character.',
 "Atop the Main Building's gold dome is a golden statue of the Virgin Mary.",
 'Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes".',
 'Next to the Main Building is the Basilica of the Sacred Heart.',
 'Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection.',
 'It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858.',
 'At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.']

In [1736]:
contextword_list = []
lemmatizer = WordNetLemmatizer()

 
for i in sentencelist:
    words = nltk.word_tokenize(i)
    #words = ([x.lower() for x in words])
    words = ([x for x in words if x not in stopword])
    words = [lemmatizer.lemmatize(x) for x in words]
    words = [[s.encode('ascii') for s in words]]
    contextword_list.append(words)

    

In [1737]:
contextword_list[0]

[['Architecturally', ',', 'school', 'Catholic', 'character', '.']]

##### Extracting answer words #####

In [1738]:
ans_list = []
for i in range(len(sqd_data.data[0]['paragraphs'][0]['qas'])): 
    l1 = sqd_data.data[0]['paragraphs'][0]['qas'][i] 

    ans_word = l1['answers'][0]['text']
    ans_word = unicodedata.normalize('NFKD', ans_word).encode('ascii','ignore')
    ans_split = ans_word.split()
    #ans_words = [x.lower() for x in ans_split]
    ans_list.append(ans_split) 
    
    

In [1739]:
ans_list

[['Saint', 'Bernadette', 'Soubirous'],
 ['a', 'copper', 'statue', 'of', 'Christ'],
 ['the', 'Main', 'Building'],
 ['a', 'Marian', 'place', 'of', 'prayer', 'and', 'reflection'],
 ['a', 'golden', 'statue', 'of', 'the', 'Virgin', 'Mary']]

##### Converting answer words to 'unicode'  for NER #####

In [1740]:
answers = []
for i in range(len(ans_list)):
    l1 = ' '.join(ans_list[i]) 
    l1 = unicode(l1,"utf-8")
    answers.append(l1) 

In [1741]:
answers

[u'Saint Bernadette Soubirous',
 u'a copper statue of Christ',
 u'the Main Building',
 u'a Marian place of prayer and reflection',
 u'a golden statue of the Virgin Mary']

In [1742]:
ans_mapping = []
ans_word = []
for i in range(len(answers)):
    doc = nlp(answers[i])
    for ent in doc.ents:
        ans_mapping.append(ent.label_)  
        ans_word.append(ent.text)
        

##### Answer mapping ###### 

In [1743]:
Answer_character_df = pd.DataFrame()
Answer_character_df['QuestionNo'] = range(len(ans_mapping))
Answer_character_df['QuestionMapping'] = ans_mapping
Answer_character_df['AnswerWord'] = ans_word
Answer_character_df['AnsWordTag'] = ans_mapping

In [1744]:
Answer_character_df

,QuestionNo,QuestionMapping,AnswerWord,AnsWordTag
0,0,PERSON,Saint Bernadette Soubirous,PERSON
1,1,NORP,Christ,NORP
2,2,FAC,the Main Building,FAC
3,3,PERSON,Marian,PERSON


###################################################################################################################

##### Co occurance matrix #####

In [1745]:
Answer_document = []
for i in range(len(Answer_character_df.index)): 
    Answer_document.append([Answer_character_df['QuestionMapping'][i],Answer_character_df['AnsWordTag'][i]]) 

In [1746]:
Answer_document

[[u'PERSON', u'PERSON'],
 [u'NORP', u'NORP'],
 [u'FAC', u'FAC'],
 [u'PERSON', u'PERSON']]

In [1747]:
Question_character_df.head()

,QuestionNO,QuestionMapping,Word,WordTag
0,0,Whom did,1858,DATE
1,0,Whom did,Lourdes France,GPE
2,1,What is,the Notre Dame Main Building,LOC
3,2,Which structure,Basilica,PERSON
4,2,Which structure,Notre Dame,GPE


In [1748]:
Question_document = []
for i in range(len(Question_character_df.index)):
    Question_document.append([Question_character_df['QuestionMapping'][i],Question_character_df['WordTag'][i]])


In [1749]:
Question_document

[['Whom did', u'DATE'],
 ['Whom did', u'GPE'],
 ['What is', u'LOC'],
 ['Which structure', u'PERSON'],
 ['Which structure', u'GPE'],
 ['What is', u'GPE'],
 ['What sits', u'FAC']]

##### joining question-tag and answer-tag #####

In [1750]:
document = Answer_document + Question_document

In [1751]:
document


[[u'PERSON', u'PERSON'],
 [u'NORP', u'NORP'],
 [u'FAC', u'FAC'],
 [u'PERSON', u'PERSON'],
 ['Whom did', u'DATE'],
 ['Whom did', u'GPE'],
 ['What is', u'LOC'],
 ['Which structure', u'PERSON'],
 ['Which structure', u'GPE'],
 ['What is', u'GPE'],
 ['What sits', u'FAC']]

In [1752]:
import numpy
import itertools
from collections import Counter
varnames = tuple(sorted(set(itertools.chain(*document))))


In [1753]:
expanded = [tuple(itertools.combinations(d, 2)) for d in document]
expanded = itertools.chain(*expanded)

# Sort the combinations so that A,B and B,A are treated the same
expanded = [tuple(sorted(d)) for d in expanded]

# count the combinations
c = Counter(expanded) 


# Create the table
table = numpy.zeros((len(varnames),len(varnames)), dtype=int)

for i, v1 in enumerate(varnames):
    for j, v2 in enumerate(varnames[i:]):        
        j = j + i 
        table[i, j] = c[v1, v2]
        table[j, i] = c[v1, v2]


In [1754]:
co_occ_matrix = pd.DataFrame(table,index = varnames,columns = varnames)
co_occ_matrix

,DATE,FAC,GPE,LOC,NORP,PERSON,What is,What sits,Which structure,Whom did
DATE,0,0,0,0,0,0,0,0,0,1
FAC,0,1,0,0,0,0,0,1,0,0
GPE,0,0,0,0,0,0,1,0,1,1
LOC,0,0,0,0,0,0,1,0,0,0
NORP,0,0,0,0,1,0,0,0,0,0
PERSON,0,0,0,0,0,2,0,0,1,0
What is,0,0,1,1,0,0,0,0,0,0
What sits,0,1,0,0,0,0,0,0,0,0
Which structure,0,0,1,0,0,1,0,0,0,0
Whom did,1,0,1,0,0,0,0,0,0,0


###################################################################################################################

##### Select answer sentence from text ##### 

* Count the common words between "Question sentence" and each "sentence"

In [1755]:
l2 = [] 
for i in range(len(Question_list)):
    l2.append(nltk.word_tokenize(Question_list[i])) 

In [1756]:
lower_qn_list = []
for i in range(len(l2)):
    lower = ([x.lower() for x in l2[i]])
    lower_qn_list.append(lower) 

In [1757]:
mn = np.zeros([len(lower_qn_list),len(sentence_list)])

In [1758]:
mn

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [1759]:
for i in range(len(lower_qn_list)):
    for j in range(len(sentence_list)):
        
        mn[i,j] = len(set(sentence_list[j][0])&set(lower_qn_list[i])) 
        

In [1760]:
mn

array([[ 0.,  2.,  0.,  0.,  0.,  5.,  1.],
       [ 0.,  2.,  3.,  2.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  3.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  1.,  0.],
       [ 0.,  2.,  2.,  2.,  0.,  0.,  1.]])

In [1761]:
pd.DataFrame(mn, index = ['Q1','Q2','Q3','Q4','Q5'],columns = ['S1','S2','S3','S4','S5','S6','S7'])  

,S1,S2,S3,S4,S5,S6,S7
Q1,0.0,2.0,0.0,0.0,0.0,5.0,1.0
Q2,0.0,2.0,3.0,2.0,0.0,0.0,1.0
Q3,0.0,0.0,0.0,3.0,1.0,0.0,0.0
Q4,0.0,0.0,0.0,0.0,1.0,1.0,0.0
Q5,0.0,2.0,2.0,2.0,0.0,0.0,1.0


###################################################################################################################

#################################################################################################################

###### PART -II #####

##### Word move Distance for extracting Answer sentence from the text #####

In [1762]:
import gensim

In [1763]:
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('/home/anjana/Downloads/GoogleNews-vectors-negative300.bin', binary=True)


##### Construction of word move distance matrix #####

In [1764]:
wmd_matrix = np.zeros([len(question_list),len(senttokenized_context)]) 
for i in range(len(question_list)):
    for j in range(len(senttokenized_context)):
        wmd_matrix[i,j] =  word2vec_model.wmdistance(question_list[i].split(),senttokenized_context[j].split()) 


##### Word mover  distance for each question across all sentences #####

* Rows indicates question number and column represents sentence number 

In [1765]:
wmd_df = pd.DataFrame(wmd_matrix) 

In [1766]:
wmd_df

,0,1,2,3,4,5,6
0,2.652292,2.631091,2.906310,2.734407,2.763077,2.120559,2.690732
1,2.678404,2.437435,2.435459,2.067550,2.542102,2.465135,2.470335
2,2.822904,2.855032,2.868838,1.851679,2.502111,2.519347,2.680416
3,2.618026,2.740448,2.880111,2.373554,2.481385,2.412599,2.576077
4,2.837293,2.707262,2.624205,1.972891,2.872521,2.674597,2.677626


##### Add a new column 'sentence' indicating the answer sentence ##### 

In [1767]:
wmd_df['sentence'] = wmd_df.T.idxmin()


In [1768]:
wmd_df 

,0,1,2,3,4,5,6,sentence
0,2.652292,2.631091,2.906310,2.734407,2.763077,2.120559,2.690732,5
1,2.678404,2.437435,2.435459,2.067550,2.542102,2.465135,2.470335,3
2,2.822904,2.855032,2.868838,1.851679,2.502111,2.519347,2.680416,3
3,2.618026,2.740448,2.880111,2.373554,2.481385,2.412599,2.576077,3
4,2.837293,2.707262,2.624205,1.972891,2.872521,2.674597,2.677626,3


##### Mapping "Question and Answers"  #####

In [1771]:
qn_sentence = []
for i in range(len(qn_ans)):
    if qn_ans[i][0] == question_list.index(question_list[i]):
        qn_sentence.append(unicode(question_list[i],"utf-8")) 
an_sentence = []
for i in range(len(qn_ans)):
    for j in range(len(senttokenized_context)):
        if qn_ans[i][1] == senttokenized_context.index(senttokenized_context[j]):
            an_sentence.append(unicode(senttokenized_context[j],"utf-8")) 

            

##### NER for "Question sentence"  and "Answer sentence" #####

In [1772]:
an_sentence_mapping = []
for i in range(len(an_sentence)):
    doc = nlp(an_sentence[i])
    for ent in doc.ents:
        an_sentence_mapping.append([i,ent.label_])  

In [1773]:
qn_sentence_mapping = []
for i in range(len(qn_sentence)):
    doc = nlp(qn_sentence[i])
    for ent in doc.ents:
        qn_sentence_mapping.append([i,ent.label_])  

##### Editing  Question words #####

- Remove stopwords like "did" ,"is", "was" from "question words".

In [1786]:
#sample done on 0th context under the title notre dame
Question_mapping = []
for i in range(len(question_mapping)):
    Question_mapping.append(question_mapping[i].split()[0])

In [1787]:
Question_mapping

['Whom', 'What', 'Which', 'What', 'What']

In [1788]:
Question_character_df = pd.DataFrame()
qn_mapping = []
for i in range(len(Question_no)):
    for j in range(len(Question_mapping)):
        if (Question_no[i]) == j:
            qn_mapping.append(Question_mapping[j])


Question_character_df['QuestionNO'] = Question_no
Question_character_df['QuestionMapping'] = qn_mapping
Question_character_df['Word'] = Qn_word
Question_character_df['WordTag'] = Qn_tag


#####  Create a co occurence matrix #####

In [1789]:
row_list = ['How many','How much','What','When','Where','Who', 'Which','Whom','PERSON',
           'NORP', 'FAC','LOC','PRODUCT', 'EVENT','WORK_OF_ART','LANGUAGE' ,'DATE' ,
           'TIME' ,'PERCENT' ,'MONEY' ,'ORDINAL','CARDINAL','GPE']

column_list = ['PERSON',
           'NORP', 'FAC','LOC','PRODUCT', 'EVENT','WORK_OF_ART','LANGUAGE' ,'DATE' ,
           'TIME' ,'PERCENT' ,'MONEY' ,'ORDINAL','CARDINAL','GPE'] 

In [1790]:
co_occ_matrix = np.zeros((len(row_list), len(column_list)), dtype = int)
co_occ_matrix

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0,

##### Creating a list of lists with question-answer NER tag #####

In [1791]:
def que_ans_combo_(question_words, ans_words):
    ques_ans_combo = []
    for i in range(len(question_words)):
        for j in range(len(ans_words)):
            ques_ans_combo.append([question_words[i],ans_words[j]])
    return(ques_ans_combo) 

##### Filling in the co occurance matrix with corresponding count #####



In [1817]:
Answer_character_df = pd.DataFrame(an_sentence_mapping,columns = ['QuestionNo','WordTag'])
Answer_character_df.head()

,QuestionNo,WordTag
0,0,GPE
1,0,GPE
2,0,PERSON
3,0,PERSON
4,0,DATE


In [1793]:

for i in range(len(Question_mapping)):

    question_words = Question_character_df[Question_character_df['QuestionNO']== i]['WordTag'].tolist() + [Question_mapping[i]] 
    ans_words = Answer_character_df[Answer_character_df['QuestionNo']==i]['WordTag'].tolist()
    combo = que_ans_combo_(question_words, ans_words)
    print(combo)
    for i, v1 in enumerate(row_list):
        for j, v2 in enumerate(column_list):
            for k in combo:
                if [v1, v2] == k:
                    co_occ_matrix[i, j]+=1
    #                     co_occ_matrix_df = pd.DataFrame(co_occ_matrix, index = row_list, columns= column_list)


[[u'DATE', u'GPE'], [u'DATE', u'GPE'], [u'DATE', u'PERSON'], [u'DATE', u'PERSON'], [u'DATE', u'DATE'], [u'GPE', u'GPE'], [u'GPE', u'GPE'], [u'GPE', u'PERSON'], [u'GPE', u'PERSON'], [u'GPE', u'DATE'], ['Whom', u'GPE'], ['Whom', u'GPE'], ['Whom', u'PERSON'], ['Whom', u'PERSON'], ['Whom', u'DATE']]
[[u'LOC', u'FAC'], [u'LOC', u'PERSON'], [u'LOC', u'ORG'], ['What', u'FAC'], ['What', u'PERSON'], ['What', u'ORG']]
[[u'PERSON', u'FAC'], [u'PERSON', u'PERSON'], [u'PERSON', u'ORG'], [u'GPE', u'FAC'], [u'GPE', u'PERSON'], [u'GPE', u'ORG'], ['Which', u'FAC'], ['Which', u'PERSON'], ['Which', u'ORG']]
[[u'GPE', u'FAC'], [u'GPE', u'PERSON'], [u'GPE', u'ORG'], ['What', u'FAC'], ['What', u'PERSON'], ['What', u'ORG']]
[[u'FAC', u'FAC'], [u'FAC', u'PERSON'], [u'FAC', u'ORG'], ['What', u'FAC'], ['What', u'PERSON'], ['What', u'ORG']]



##### Co occurence matrix for the entire  Zerot context under the title notre dame  #####
##### Row corresponds to "Question" type and NER tags #####
##### Column corresponds to "Answer" NER #####

In [1794]:
co_occ_matrix_df = pd.DataFrame(co_occ_matrix, index = row_list, columns= column_list)
co_occ_matrix_df

,PERSON,NORP,FAC,LOC,PRODUCT,EVENT,WORK_OF_ART,LANGUAGE,DATE,TIME,PERCENT,MONEY,ORDINAL,CARDINAL,GPE
How many,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
How much,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
What,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0
When,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Where,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Who,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Which,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
Whom,2,0,0,0,0,0,0,0,1,0,0,0,0,0,2
PERSON,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
NORP,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


##### Graphical representation of co occurence matrix #####

In [1819]:
co_occ_matrix_df['Tags'] = co_occ_matrix_df.index


In [1816]:
melt_co_occ_matrix_df = pd.melt(co_occ_matrix_df,id_vars = ['Tags']) 
melt_co_occ_matrix_df.head()

,Tags,variable,value
0,How many,PERSON,0
1,How much,PERSON,0
2,What,PERSON,3
3,When,PERSON,0
4,Where,PERSON,0


In [1815]:
import plotly.plotly as py
import plotly.graph_objs as go

trace = go.Heatmap(z=melt_co_occ_matrix_df['value'],
                   x=melt_co_occ_matrix_df['Tags'] ,
                   y=melt_co_occ_matrix_df['variable'])
data=[trace]
py.iplot(data, filename='labelled-heatmap')  